In [1]:
import pandas as pd
from tqdm.auto import tqdm
import sys
sys.path.append("../")
from src.data.make_dataset import STORM_EVENTS_CSV_PATH, INTERIM_DATA_DIR

In [2]:
storm_events_csv_files = list(STORM_EVENTS_CSV_PATH.glob("*.csv"))
len(storm_events_csv_files)

216

In [3]:
event_detail_files = [x for x in storm_events_csv_files if "_details-" in str(x)]
event_fatality_files = [x for x in storm_events_csv_files if "_fatalities-" in str(x)]
event_location_files = [x for x in storm_events_csv_files if "_locations-" in str(x)]
other_files = [
    x for x in storm_events_csv_files if x not in \
    event_detail_files + event_fatality_files + event_location_files]
len(other_files)

0

In [4]:
pd.read_csv(event_detail_files[0], nrows=0)

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE


In [5]:
# detect headers
event_file_header_baseline = pd.read_csv(event_detail_files[0], nrows=0).columns
event_fatality_file_header_baseline = pd.read_csv(event_fatality_files[0], nrows=0).columns
event_location_file_header_baseline = pd.read_csv(event_location_files[0], nrows=0).columns


def headers_match(columns, csv_file_path) -> bool:
    return columns.intersection(pd.read_csv(csv_file_path, nrows=0).columns).size == len(columns)
    

assert all([headers_match(event_file_header_baseline, x) for x in event_detail_files])
assert all([headers_match(event_fatality_file_header_baseline, x) for x in event_fatality_files])
assert all([headers_match(event_location_file_header_baseline, x) for x in event_location_files])
print("Wow, all files have the same header! Nice")

Wow, all files have the same header! Nice


In [9]:
def combine_files_from_list(write_path, file_list, chunk_size=1000):
    # Write header
    pd.read_csv(file_list[0], nrows=0).to_csv(write_path, header=True, index=False)
    for file_path in tqdm(file_list):
        for chunk in pd.read_csv(file_path, chunksize=chunk_size):
            chunk.to_csv(write_path, mode='a', header=False, index=False)

In [10]:
STORM_EVENTS_COMBINED_PATH = INTERIM_DATA_DIR / "StormEventsCombined"
if not STORM_EVENTS_COMBINED_PATH.exists():
    STORM_EVENTS_COMBINED_PATH.mkdir()

In [11]:
combine_files_from_list(
    STORM_EVENTS_COMBINED_PATH / "StormEvents_details-all.csv", 
    event_detail_files
)

  0%|          | 0/72 [00:00<?, ?it/s]

In [12]:
combine_files_from_list(
    STORM_EVENTS_COMBINED_PATH / "StormEvents_fatalities-all.csv", 
    event_fatality_files
)

  0%|          | 0/72 [00:00<?, ?it/s]

In [13]:
combine_files_from_list(
    STORM_EVENTS_COMBINED_PATH / "StormEvents_locations-all.csv", 
    event_location_files
)

  0%|          | 0/72 [00:00<?, ?it/s]